# Semesterprojekt: Retourenvorhersage
Data Mining & Grundlagen Maschinelles Lernen 1, Wintersemester 2022/23

**Gruppe:**

|Paul|Antonio|Tim|Flo|
|---|---|---|---|
|hopa1029|coan1014|ehti1011|klfl1012|
80001|       |80055|79679|

**Gliederung:**
1. Data Cleaning, Feature Engineering und Funktionen
   - 1.1 Data Cleaning
  
   - 1.2 Featrue Engineering
  
   - 1.3 Funktionen
   
2. Klassifikaitonsmodelle mit Hyperparameteroptimierung:
   - 2.1 Logistische Regression 
  
   - 2.2 Random Forest
  
   - 2.3 Gradient Boosting

   - 2.4 Super Vector Machine

                  
3. Evaluation mit ROC Plot 

4. Aufgabe 5

**Datensatz und Import:** 

Der Datensatz beschreibt 75,007 bestellte Artikel, von denen bekannt ist, ob sie zurückge
sendet wurden oder nicht.  
Für einen bestellten Artikel (1 Zeile) existieren folgende Felder in den Daten:

- order_item_id: ID des bestellten Artikels
- order_date: Datum der Bestellung
- delivery_date: Lieferdatum des Artikels
- item_id: ID des Artikels
- item_size: Größe des Artikels
- item_color: Farbe des Artikels
- brand_id: Hersteller ID
- item_price: Preis des Artikels
- user_id: ID des Nutzers
- user_title: Titel des Nutzers
- user_dob: Geburtsdatum des Nutzers
- user_state: Bundesland, in dem der Nutzer wohnt
- user_reg_date: Datum, an dem sich der Nutzer registriert hat
- return: Wurde der Artikel zurückgeschickt (1) oder nicht (0)

In [1]:
import pandas as pd
df = pd.read_csv("retouren.csv")

In [2]:
df.head()

,order_item_id,order_date,delivery_date,item_id,item_size,item_color,brand_id,item_price,user_id,user_title,user_dob,user_state,user_reg_date,return
0,1,2016-06-22,2016-06-27,643,38,navy,30,49.90,30822,Mrs,1969-04-17,Saxony,2016-06-23,0
1,2,2016-06-22,NaN,337,152,grey,30,19.95,30822,Mrs,1969-04-17,Saxony,2016-06-23,0
2,3,2016-06-22,2016-06-27,270,xxl,grey,49,79.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1
3,4,2016-06-22,2016-06-27,142,xxl,grey,49,99.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,0
4,5,2016-06-22,2016-06-27,561,xxl,grey,3,14.90,30823,Mrs,1970-04-22,Baden-Wuerttemberg,2015-03-15,1


## 1. Data Cleaning, Feature Engineering und Funktionen

### 1. Data Cleaning

**Anpassen der Datentypen**

In [3]:
df["user_dob"] = pd.to_datetime(df.user_dob)
df["user_reg_date"] = pd.to_datetime(df.user_reg_date)
df["order_date"] = pd.to_datetime(df.order_date)
df["delivery_date"] = pd.to_datetime(df.delivery_date)

**Löschen der Datensätze mit Preis von 999**

In [4]:
df = df.drop(df[df.item_price == 999].index)

**Löschen der Datensätze wo das Lieferdatum kleiner dem Bestelldatum**

In [5]:
df = df.drop(df[df.delivery_date < df.order_date].index)

**Anpassen der Dateninkonsistenzen bei item_color**

In [6]:
df = df.drop(df[df.item_color == "?"].index)

In [7]:
df.loc[df.item_color == "brwon","item_color"] = "brown"
df.loc[df.item_color == "blau","item_color"] = "blue"
df.loc[df.item_color == "dark oliv","item_color"] = "dark olive"

**Löschen der Datensätze ohne delivery_date** 

In [11]:
df.drop(df[df.delivery_date.isna()].index, inplace=True)

### 2.2 Feature Engineering

**Datensatz mit neuen Features:**

Der Datensatz wurde für die Modelle um folgenden Features erweitert:

- age: Alter der User zum Zeitpunkt der Bestellung, Differenz zwischen order_date und user_dob
- acc_age: Accountalter des Nutzers zum Zeitpunkt der Bestellung, (-) wenn nach der Bestellung regisrtiert
- maxprice: Der Maximalpreis eines Produktes
- markdown: Ist der Artikel vergünstig (1) oder nicht (0) 

In [8]:
df["age"] = df.order_date.dt.year - df.user_dob.dt.year -((df.order_date.dt.date)<(df.user_dob.dt.date))

In [9]:
df["acc_age"] = df.order_date - df.user_reg_date 
df["acc_age"] = df.acc_age.dt.days

In [10]:
df_mark = df.groupby(["item_id","item_size","item_color"]).agg(maxprice=("item_price","max")).reset_index()
df = pd.merge(df,df_mark)
df["markdown"] = 0
df.loc[df.item_price < df.maxprice,"markdown"] = 1

In [1]:
df.head()

NameError: name 'df' is not defined

### 3. Funktionen 

## 2. Klassifikaitonsmodelle mit Hyperparameteroptimierung

### 2.1 Logisitsche Regression

### 2.2 Random Forest

### 2.3 Gradient Boosting

### 2.4 Super Vector Machine

## 3. Evaluation mit ROC Plot

## 4. Aufgabe 5